# Test Service

Intended to test the service.py evaluator.
Runs the service.py and a simple client.



# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import PIL
from flatland.utils.rendertools import RenderTool
import imageio
import os

In [4]:
from IPython.display import clear_output
from IPython.core import display
import ipywidgets as ipw
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

In [12]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.line_generators import sparse_line_generator
from flatland.envs.malfunction_generators import malfunction_from_file, no_malfunction_generator
from flatland.envs.rail_generators import rail_from_file
from flatland.core.env_observation_builder import DummyObservationBuilder
from flatland.envs.persistence import RailEnvPersister
from flatland.evaluators.client import FlatlandRemoteClient, TimeoutException

In [14]:
import pickle
import redis
import subprocess as sp
import shlex
import time
import pkg_resources as pr
import importlib_resources as ir
import sys, os
import pandas as pd

### Find the real path of the `env_data` package (should be copied by tox)

In [17]:
with ir.path("env_data.tests", "test_001.pkl") as oPath:
    sPath = oPath
print(type(sPath), sPath)

<class 'pathlib.WindowsPath'> c:\users\u216993\.conda\envs\flatland3-rl\lib\site-packages\env_data\tests\test_001.pkl


In [18]:
sDirRoot = "/" + "/".join(sPath.parts[1:-1] + ("service_test",""))
sDirRoot

'/users/u216993/.conda/envs/flatland3-rl/lib/site-packages/env_data/tests/service_test/'

### Clear any old redis keys

In [19]:
oRedis = redis.Redis()

In [20]:
lKeys = oRedis.keys("flatland*")
lKeys

ConnectionError: Error 10061 connecting to localhost:6379. Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte.

In [21]:
for sKey in lKeys:
    print("Deleting:", sKey)
    oRedis.delete(sKey)

NameError: name 'lKeys' is not defined

### Remove `/tmp/output.csv`

In [ ]:
!rm -f /tmp/output.csv

### kill any old `service.py` process

In [ ]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [ ]:
osEnv2 = os.environ.copy()

### Timeouts copied from service.py

In [ ]:
#MAX_SUCCESSIVE_TIMEOUTS = int(os.getenv("FLATLAND_MAX_SUCCESSIVE_TIMEOUTS", 10))

# 8 hours (will get debug timeout from env variable if applicable)
#OVERALL_TIMEOUT = int(os.getenv(
#    "FLATLAND_OVERALL_TIMEOUT",
#    8 * 60 * 60))

# 10 mins
#INTIAL_PLANNING_TIMEOUT = int(os.getenv(
#    "FLATLAND_INITIAL_PLANNING_TIMEOUT",
#    10 * 60))

# 10 seconds
#PER_STEP_TIMEOUT = int(os.getenv(
#    "FLATLAND_PER_STEP_TIMEOUT",
#    10))

# 5 min - applies to the rest of the commands
#DEFAULT_COMMAND_TIMEOUT = int(os.getenv(
#    "FLATLAND_DEFAULT_COMMAND_TIMEOUT",
#    5 * 60))

### Set some short timeouts for testing

In [ ]:
osEnv2["FLATLAND_OVERALL_TIMEOUT"]="10"
osEnv2["FLATLAND_PER_STEP_TIMEOUT"] = "2"
osEnv2["FLATLAND_MAX_SUCCESSIVE_TIMEOUTS"] = "2"

### Create the python command for `service.py`

In [ ]:
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --mergeDir ./tmp/merge --actionDir ./tmp/actions --pickle --missingOnly"
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --pickle" # --verbose"
sCmd = f"python -m flatland.evaluators.service --test_folder {sDirRoot} --pickle" # --verbose"
lsCmd = shlex.split(sCmd)
print(sCmd)
print(lsCmd)

### Run the command with Popen (output goes to jupyter stdout not notebook)

In [ ]:
oPipe = sp.Popen(lsCmd, env=osEnv2)

In [ ]:
oPipe.poll()

In [ ]:
#oFRC = FlatlandRemoteClient(test_envs_root="../env_data/tests/service_test/", verbose=False, use_pickle=True)
oFRC = FlatlandRemoteClient(test_envs_root=sDirRoot, verbose=False, use_pickle=True)

In [ ]:
#env, env_dict = RailEnvPersister.load_new("../env_data/tests/service_test/Test_0/Level_0.pkl") # env_file)
env, env_dict = RailEnvPersister.load_new(f"{sDirRoot}/Test_0/Level_0.pkl") # env_file)
ldActions = env_dict["actions"]

In [ ]:
def expert_controller(obs, _env):
    return ldActions[_env._elapsed_steps]

def random_controller(obs, _env):
    dAct = {}
    for iAg in range(len(_env.agents)):
        dAct[iAg] = np.random.randint(0, 5)
    return dAct

In [ ]:
oObsB = DummyObservationBuilder()

In [ ]:
oObsB.get()

In [ ]:
def run_submission(slow_ep=1, delay=2):
    episode = 0
    obs = True
    while obs:
        obs, info = oFRC.env_create(obs_builder_object=oObsB)
        if not obs:
            print("null observation - all envs completed!")
            break
        print(f"Episode : {episode}")
        

        print(oFRC.env.dones['__all__'])

        while True:
            if episode < 3:
                action = expert_controller(obs, oFRC.env)
            else:
                action = random_controller(obs, oFRC.env)
            
            time_start = time.time()
            
            if (episode == slow_ep) and (oFRC.env._elapsed_steps > 10):
                time.sleep(2)
                
            try:
                observation, all_rewards, done, info = oFRC.env_step(action)
                time_diff = time.time() - time_start
                print(".", end="")
                if done['__all__']:
                    print("\nCompleted Episode : ", episode)
                    print("Reward : ", sum(list(all_rewards.values())))
                    break
            except TimeoutException as err:
                print("Timeout: ", err)
                break
            
        episode += 1
        
    print(f"Evaluation Complete - episodes={episode} - send submit message...")
    print(oFRC.submit())
    print("All done.")
        

In [ ]:
try:
    run_submission()
except TimeoutException as timeoutException:
    print("Timed out.")
    print(timeoutException)
    

### Kill the evaluator process we started earlier

In [ ]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [ ]:
pd.read_csv("/tmp/output.csv").T